### Main Libraries Needed

In [1]:
#Import of main libraries

%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np 
from __future__ import division, print_function
from pandas import read_csv
from pandas import read_excel
from pandas import DataFrame
from pandas import ExcelWriter
from pandas import ExcelFile

#Import of the pyomo module
from pyomo.environ import *

### Importing and creating necessary Data Frames

In [14]:
#Import Bids, Lanes, and Carrier DF's
bidsDf = read_csv("Germany Bids.csv")
bidsDf = bidsDf.drop("Unnamed: 0", axis = 1)
lanesDf = read_csv("Germany Lanes.csv")
lanesDf = lanesDf.drop("Unnamed: 0", axis = 1)
CarriersDf = read_csv("Carriers.csv")
CarriersDf = CarriersDf.drop("Unnamed: 0", axis = 1)

In [16]:
#Create data frames for delta matrix
deltaDf = DataFrame(np.zeros((len(lanesDf.index), len(bidsDf.index))))
for bid in bidsDf.index:
    deltaDf.at[bidsDf.loc[bid,'Cluster Lot Index'] - 1, bid] = 1

In [18]:
# Create the eta DF
etaDf = DataFrame(np.zeros((len(CarriersDf.index), len(bidsDf.index))))
for bid in bidsDf.index:
    etaDf.at[bidsDf.loc[bid,'Carrier Index'] - 1, bid] = 1

In [19]:
import pandas as pd

In [20]:
Mc = pd.Series(np.sum(etaDf, axis = 1))

In [21]:
Mc

0     25.0
1     28.0
2     24.0
3     27.0
4      9.0
5     27.0
6      0.0
7     28.0
8     27.0
9     25.0
10     1.0
11     1.0
12    26.0
13    11.0
14     0.0
15    19.0
dtype: float64

### Create the necessary dictionaries

In [22]:
carriers = Mc.to_dict()

bidValues = dict()
for bid in bidsDf.index:
    bidValues[bid] = bidsDf.loc[bid, 'Cost']

demandValues = dict()
for lane in lanesDf.index:
    demandValues[lane] = 1

delta = dict()
for lane in lanesDf.index:
    for bid in bidsDf.index:
        delta[(lane,bid)] = deltaDf.loc[lane,bid]

eta = dict()
for c in CarriersDf.index:
    for bid in bidsDf.index:
        eta[(c,bid)] = etaDf.loc[c,bid]


In [23]:
# Create the Concrete Model
model = ConcreteModel()

model.numBids = len(bidsDf.index)

model.numItems = len(lanesDf.index)

model.numCarriers = len(CarriersDf.index)

model.BIDS = Set(initialize = bidsDf.index.values)

model.LANES = Set(initialize = lanesDf.index.values)

model.CARRIERS = Set( initialize = CarriersDf.index.values)

model.M = Param(model.CARRIERS, initialize = carriers)

model.bidValue = Param(model.BIDS, initialize = bidValues, doc='Value of each bid in the program')

model.demand = Param(model.LANES, initialize = demandValues, doc='Total demand on each lane')

# Below is where I made the suggested work around from https://stackoverflow.com/questions/45616967/pyomo-valueerror-invalid-constraint-expression

model.delta = Param(model.LANES, model.BIDS, initialize= delta, doc='delta gives information regarding which lanes are in a bid package')#,mutable = True)

model.eta = Param(model.CARRIERS, model.BIDS, initialize= eta, doc='eta gives information regarding which carriers are in a bid package', mutable = True)

model.x = Var(model.BIDS, domain = Binary, doc='Decision variable for each bid in the program')

model.z = Var(model.CARRIERS, domain = Binary, doc='Decision variable for each carrier in the program')

model.maxCarriers = 5
model.minCarriers = 1

### Set Model Obj and Constraints

In [24]:
def obj_expression(model):
    return sum(model.bidValue[i]*model.x[i] for i in model.BIDS)
model.OBJ = Objective(rule=obj_expression, sense=minimize, doc='Objective function definition')

In [25]:
def constraint_rule(model, l):
    return sum(model.delta[l,b]*model.x[b] for b in model.BIDS) <= model.demand[l]
model.xConstraint = Constraint(model.LANES, rule=constraint_rule)

def demand_constraint_rule(model):
    return sum(model.x[b] for b in model.BIDS) >= model.numItems
model.demandConstraint = Constraint(rule=demand_constraint_rule)

def constraint2_rule(model, k):
    return sum(model.eta[k,b]*model.x[b] for b in model.BIDS) - model.M[k]*model.z[k]<=0
model.upperBoundConstraint = Constraint(model.CARRIERS, rule=constraint2_rule)

def constraint3_rule(model, k):
    return model.z[k] - sum(model.eta[k,b]*model.x[b] for b in model.BIDS)<=0
model.lowerBoundConstraint = Constraint(model.CARRIERS, rule=constraint3_rule)

def constraint4_rule(model):
    return sum(model.z[i] for i in model.CARRIERS)<=model.maxCarriers
model.zConstraint = Constraint(rule=constraint4_rule)

def constraint5_rule(model):
    return sum(model.z[i] for i in model.CARRIERS)>=model.minCarriers
model.zConstraint2 = Constraint(rule=constraint5_rule)

### Run the Model

In [26]:
def pyomo_postprocess(options=None, instance=None, results=None):
    model.x.display()

In [27]:
from pyomo.opt import SolverFactory
import pyomo.environ
opt = SolverFactory("glpk")
%timeit results = opt.solve(model)

192 ms ± 8.54 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [28]:
results = opt.solve(model)
model.solutions.store_to(results)
print(results)


Problem: 
- Name: unknown
  Lower bound: 85621.2
  Upper bound: 85621.2
  Number of objectives: 1
  Number of constraints: 64
  Number of variables: 295
  Number of nonzeros: 1175
  Sense: minimize
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 81
      Number of created subproblems: 81
  Error rc: 0
  Time: 0.0724782943725586
Solution: 
- number of solutions: 1
  number of solutions displayed: 1
- Gap: 0.0
  Status: optimal
  Message: None
  Objective:
    OBJ:
      Value: 85621.20000000001
  Variable:
    x[0]:
      Value: 1
    x[102]:
      Value: 1
    x[10]:
      Value: 1
    x[111]:
      Value: 1
    x[123]:
      Value: 1
    x[135]:
      Value: 1
    x[144]:
      Value: 1
    x[156]:
      Value: 1
    x[168]:
      Value: 1
    x[179]:
      Value: 1
    x[188]:
      Value: 1
    x[197]:
      Value: 1
    x[208]:
      Value: 1
    x[20]:
      Value: 1
    x[217]:
      Value: 1
    x

In [29]:
results.write()
print("\nDisplaying Solution\n" + '-'*60)
pyomo_postprocess(None, model, results)

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 85621.2
  Upper bound: 85621.2
  Number of objectives: 1
  Number of constraints: 64
  Number of variables: 295
  Number of nonzeros: 1175
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 81
      Number of created subproblems: 81
  Error rc: 0
  Time: 0.0724782943725586
# ----------------------------------------------------------
#   Solution Information
# --------------------------

In [0]:
#model.pprint()

In [30]:
winningBids = []
index = 0
bidNum = 0
for bids in range(len(bidsDf.index)):
    if model.x[bids].value > 0:
        winningBids.append(bidNum)
    bidNum += 1
    index += 1

winningBidsDf = bidsDf.iloc[winningBids]
winningBidsDf.head()

,Bid,Carrier,Lot,Lot#,Cost,Rank,Lowest Bid,Delta vs Lowest,Shipments per Lot,Site Cluster,Carrier Index,Cluster Lot Index
0,INTT@ISO_153,INTT,ISO_153,153,3546.0,1,3546.0,0.0,1,Germany,6,1
10,INTT@ISO_154,INTT,ISO_154,154,2971.2,2,2930.4,40.8,33,Germany,6,2
20,INTT@ISO_155,INTT,ISO_155,155,3561.6,1,3561.6,0.0,1,Germany,6,3
26,ITLW@ISO_156,ITLW,ISO_156,156,3007.2,2,2829.6,177.6,48,Germany,5,4
36,DNHG@ISO_157,DNHG,ISO_157,157,3362.4,1,3362.4,0.0,26,Germany,3,5


In [31]:
winningBidsDf

,Bid,Carrier,Lot,Lot#,Cost,Rank,Lowest Bid,Delta vs Lowest,Shipments per Lot,Site Cluster,Carrier Index,Cluster Lot Index
0,INTT@ISO_153,INTT,ISO_153,153,3546.0,1,3546.0,0.0,1,Germany,6,1
10,INTT@ISO_154,INTT,ISO_154,154,2971.2,2,2930.4,40.8,33,Germany,6,2
20,INTT@ISO_155,INTT,ISO_155,155,3561.6,1,3561.6,0.0,1,Germany,6,3
26,ITLW@ISO_156,ITLW,ISO_156,156,3007.2,2,2829.6,177.6,48,Germany,5,4
36,DNHG@ISO_157,DNHG,ISO_157,157,3362.4,1,3362.4,0.0,26,Germany,3,5
46,LXSG@ISO_158,LXSG,ISO_158,158,3225.6,1,3225.6,0.0,2,Germany,8,6
58,DNHG@ISO_159,DNHG,ISO_159,159,3024.0,2,2880.0,144.0,10,Germany,3,7
69,LXSG@ISO_160,LXSG,ISO_160,160,3660.0,3,3444.0,216.0,1,Germany,8,8
74,INTT@ISO_161,INTT,ISO_161,161,4382.4,1,4382.4,0.0,53,Germany,6,9
82,INTT@ISO_162,INTT,ISO_162,162,3344.4,1,3344.4,0.0,1,Germany,6,10


In [0]:
Total = sum(winningBidsDf['Cost']*winningBidsDf['Shipments per Lot'])
Total

In [32]:
winningBidsDf.to_csv('Germany Carrier Constraint WinningBids.csv')